## ROI clustering by size (area)

### Load the romero.gateway library

In [ ]:
library(romero.gateway)

### Connect to server

In [ ]:
server <- OMEROServer(host = 'eel.openmicroscopy.org', username='username', password='password', port= 4064L)
server <- connect(server)

### Get the dataset

Using the search functionility:

In [ ]:
searchResult <- searchFor(server, Dataset, 'Name', 'CDK5RAP2-C')
dataset <- searchResult[[1]]
paste('Dataset:', dataset@dataobject$getName(), 'loaded.')

### Get the annotations file

In particular the file Id is needed to load the OMERO HDF table

In [ ]:
annos <- getAnnotations(server, 'DatasetData', getOMEROID(dataset), nameFilter = 'Test_Table')
annotationFileID = as.integer(annos$FileID)
paste('Annotation file id:', annotationFileID)

### Check that we can load the annotations as dataframe

In [ ]:
describeDataframe(dataset, annotationFileID)

### Load the table as R dataframe

Only load the relevant data, i. e. columns 2 (Shape-id) and 6 (Area)

In [ ]:
df <- loadDataframe(dataset, annotationFileID, columns=c(2, 6))
df

### Do some analysis

#### Perform kmeans clustering and add the information to the dataframe:

In [ ]:
fit <- kmeans(sqrt(df$Area), 2)
df <- data.frame(df, fit$cluster)
names(df)[[3]] <- 'Cluster'
df

#### Plot the data

In [ ]:
plot(df$Area, ylab='Area')
text(df$Area, labels=df$Cluster, cex= 0.5, pos=3)

### Save the results back to OMERO

#### The plot

Generate the plot again, but this time save as png file and then attach it to the dataset.

In [ ]:
tmpfile <- "/tmp/Cluster_Plot.png"
png(tmpfile)
plot(df$Area, ylab='Area')
text(df$Area, labels=df$Cluster, cex= 0.5, pos=3)
dev.off()
invisible(attachFile(dataset, tmpfile))

(Note: Wrapped into invisible(...) statement to suppress unnecessary output)

#### The R dataframe

This is also stored as OMERO HDF table.

In [ ]:
invisible(attachDataframe(dataset, df, "KMeans_Clusters"))

Alternatively as CSV:

In [ ]:
tmpfile <- "/tmp/KMeans_Clusters.csv"
write.csv(df, file = tmpfile)
invisible(attachFile(dataset, tmpfile))

### Disconnect from the server again:

In [ ]:
invisible(disconnect(server))